# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 05_tools_context_demo.ipynb

This notebook demonstrates how to use the **Drop-In Knowledge Runtime**
with a **tools orchestration layer**, on top of:

- conversational memory (chat history),
- optional RAG (attachments ingested into a vector store),
- optional live web search context.

The focus of this notebook is to show **how tools are integrated and used
in a ChatGPT-like flow**, while all other configuration (LLM, embeddings,
vector store, web search) is pushed into a single compact setup cell.


## 1. Imports and environment setup

In this section we will:

- configure the Python path so the `intergrax` package can be imported from the repo,
- load environment variables (API keys, etc.),
- import the core building blocks used by the Drop-In Knowledge Runtime.

All non-tool configuration (LLM, embeddings, vector store, web search) will be
initialized later in a **single compact config cell**, so that this notebook
can stay focused on how the tools layer is integrated into the ChatGPT-like flow.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
from dotenv import load_dotenv

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
load_dotenv()


from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

# --- Compact runtime configuration setup (all non-tools components) ---

from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import RuntimeEngine
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig



# Core runtime configuration (LLM + embeddings + vector store + web search)
# -----------------------------------------------------------------------------

# Session store – simple in-memory storage for chat messages & metadata.
session_manager = SessionManager(
   storage=InMemorySessionStorage() 
)

# 2.2 LLM adapter – here we use Ollama through the LangChain adapter.
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)


# RuntimeConfig – single source of truth for drop-in knowledge runtime.
config = RuntimeConfig(
    # LLM & embeddings & vector store
    llm_adapter=llm_adapter,

    # RAG settings (enabled to allow attachment-based context, as in 03 demo)
    enable_rag=False,

    # Web search settings – THIS is the feature under test in this notebook.
    enable_websearch=False,

    # Tools / memory – kept off here to isolate the web search behavior.    
    enable_long_term_memory=False,
    enable_user_profile_memory=True,

    # Optional tenant / workspace (can be overridden per request)
    tenant_id="demo-tenant",
    workspace_id="demo-workspace",

    tools_agent=None,            # set later
    tools_mode="auto"            # placeholder
)

context = RuntimeContext(
    config=config,
    session_manager=session_manager,
)

# Drop-in knowledge runtime – chat engine with RAG + web search.
runtime = RuntimeEngine(context=context)

print("RuntimeConfig ready.")




RuntimeConfig ready.


### 2.1 Defining tools using the Intergrax tools framework

Instead of creating custom ad-hoc classes, we will reuse the existing
Intergrax tools stack:

- `ToolBase` & `ToolRegistry` for tool definitions and registration,
- `IntergraxToolsAgent` as the planner/controller,
- `ToolsAgentConfig` for agent behavior,
- `IntergraxConversationalMemory` to keep dialogue context across calls.

We will:

1. Implement two demo tools (`WeatherTool`, `CalcTool`) exactly like in the
   standalone tools notebook.
2. Register them in a `ToolRegistry`.
3. Create an `IntergraxToolsAgent` instance that uses an Ollama-based LLM.
4. Attach this agent to `RuntimeConfig.tools_agent` so that the
   Drop-In Knowledge Runtime can orchestrate tools in a ChatGPT-like flow.


In [ ]:
# --- Tools definitions and IntergraxToolsAgent wiring ---

from typing import Any, Dict, Optional

from pydantic import BaseModel, Field

from intergrax.llm_adapters.llm_usage_track import LLMUsageTracker
from intergrax.memory.conversational_memory import ConversationalMemory
from intergrax.tools.tools_agent import ToolsAgent, ToolsAgentConfig
from intergrax.tools.tools_base import ToolRegistry, ToolBase


# ---------- WEATHER TOOL ----------

class WeatherArgs(BaseModel):
    """
    Argument schema for the weather tool.

    Fields:
      - city: name of the city for which we want to fetch weather information.
    """
    city: str = Field(..., description="City name, e.g. 'Warsaw'")


class WeatherTool(ToolBase):
    """
    Simple demo weather tool.

    Responsibilities:
      - Accept a city name.
      - Return mock current weather information for that city.

    NOTE:
      In a real implementation this would call an external weather API.
    """
    name = "get_weather"
    description = (
        "Returns current weather for a city. "
        "Use only for queries about weather, temperature or conditions in a city."
    )
    schema_model = WeatherArgs

    def run(self, 
            run_id:Optional[str] = None,
            llm_usage_tracker: Optional[LLMUsageTracker] = None,
            **kwargs) -> Any:
        city = kwargs["city"]
        # Demo output (static, not calling any real API).
        return {
            "city": city,
            "tempC": 12.3,
            "summary": "partly cloudy (demo only, not real data)",
        }


# ---------- CALCULATOR TOOL ----------

class CalcArgs(BaseModel):
    """
    Argument schema for the calculator tool.

    Fields:
      - expression: a simple arithmetic expression with numbers and operators.
    """
    expression: str = Field(
        ...,
        description=(
            "A safe arithmetic expression to evaluate, "
            "e.g. '235*17', '2*(3+4)'. No variables."
        ),
    )


class CalcTool(ToolBase):
    """
    Basic arithmetic calculator tool.

    Responsibilities:
      - Validate a user-provided expression.
      - Evaluate it using a restricted environment.
      - Return both the expression and numeric result.
    """
    name = "calc_expression"
    description = (
        "Safely evaluates a basic arithmetic expression (integers/floats, + - * / parentheses). "
        "Use for math and calculation questions."
    )
    schema_model = CalcArgs

    def run(self, 
            run_id:Optional[str] = None,
            llm_usage_tracker: Optional[LLMUsageTracker] = None,
            **kwargs) -> Any:
        expr = kwargs["expression"]

        # Minimal, safe evaluation – only allow basic numeric and operator characters.
        allowed = "0123456789.+-*/() "
        if not all(ch in allowed for ch in expr):
            raise ValueError("Unsupported characters in expression.")

        # Perform the calculation in a restricted environment.
        try:
            result = eval(expr, {"__builtins__": {}}, {})
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")

        return {"expression": expr, "result": result}


# ---------- TOOLS AGENT SETUP ----------

# Conversational memory shared across tool-driven interactions.
memory = ConversationalMemory()

# Registry holding all available tools for the agent.
tools_registry = ToolRegistry()
tools_registry.register(WeatherTool())
tools_registry.register(CalcTool())

# LLM used as the planner / controller for tools.
tools_llm = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

tools_agent = ToolsAgent(
    llm=tools_llm,
    tools=tools_registry,
    memory=memory,
    config=ToolsAgentConfig(),
    verbose=True,
)

# Attach tools agent to the Drop-In Knowledge runtime configuration.
config.tools_agent = tools_agent
config.tools_mode = "auto"

print("IntergraxToolsAgent attached to Drop-In Knowledge Runtime.")


IntergraxToolsAgent attached to Drop-In Knowledge Runtime.


## 3. End-to-end Drop-In Runtime demo with tools

Now that:

- the runtime is configured (LLM + RAG + web search),
- tools are defined and registered (`WeatherTool`, `CalcTool`),
- `IntergraxToolsAgent` is attached to `RuntimeEngine.tools_agent`,

we can run a **ChatGPT-like interaction** through the
`DropInKnowledgeRuntime`.

In this section we will:

1. Define a small helper function `ask_with_tools(...)` that:
   - builds a `RuntimeRequest`,
   - sends it to the runtime (`ask_sync`),
   - prints the final answer and key debug information
     (route, tools usage, tool call traces).
2. Manually call this helper with different prompts:
   - a weather question → `get_weather`,
   - a math question → `calc_expression`,
   - a mixed question where the LLM can decide whether tools are needed.

This mirrors how tools are orchestrated in a ChatGPT-like flow:
runtime decides when to call the tools agent, then the LLM integrates
tool outputs into the final answer.


In [ ]:
# --- Async helper for end-to-end testing of Drop-In Runtime with tools ---

from pprint import pprint
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


async def ask_with_tools(
    message: str,
    *,
    user_id: str = "demo-user-tools",
    session_id: str = "demo-session-tools",
):
    """
    High-level async helper for interacting with the Drop-In Knowledge Runtime
    in a ChatGPT-like way, with tools enabled (Jupyter / notebook friendly).

    Steps:
      1. Build a RuntimeRequest with the provided message.
      2. Await runtime.ask(request).
      3. Print:
         - final model answer,
         - routing info (which layers were used),
         - tools debug trace,
         - structured tool call summaries.
    """
    request = RuntimeRequest(
        user_id=user_id,
        session_id=session_id,
        message=message,
    )

    answer = await runtime.run(request)

    print("=== ANSWER ===")
    print(answer.answer)

    print("\n=== ROUTE INFO ===")
    pprint(answer.route)

    print("\n=== TOOLS DEBUG TRACE ===")
    tools_debug = answer.debug_trace.get("tools")
    if tools_debug is not None:
        pprint(tools_debug)
    else:
        print("No tools-specific debug information.")

    print("\n=== TOOL CALLS (SUMMARY) ===")
    if answer.tool_calls:
        pprint(answer.tool_calls)
    else:
        print("No tool calls were executed.")

    return answer


print("async ask_with_tools helper is ready.")


async ask_with_tools helper is ready.


In [5]:
# Example 1: should trigger weather tool
await ask_with_tools("What is the weather and temperature in Warsaw?")

# Example 2: should trigger calculator tool
await ask_with_tools("Calculate 235*17 and give me only the numeric result.")

# Example 3: non-tool / mixed question
await ask_with_tools("Explain how tools are integrated inside this runtime.")


[intergraxToolsAgent] Iteration 1 (planner)
[intergraxToolsAgent] Calling tool: get_weather({'city': 'Warsaw'})
[intergraxToolsAgent] Iteration 2 (planner)
=== ANSWER ===
The current weather in Warsaw is partly cloudy with a temperature of 12.3 degrees Celsius.

=== ROUTE INFO ===
RouteInfo(used_rag=False,
          used_websearch=False,
          used_tools=True,
          used_long_term_memory=False,
          used_user_profile=True,
          strategy='llm_with_tools',
          extra={})

=== TOOLS DEBUG TRACE ===
{'agent_answer_preview': 'The current weather in Warsaw is partly cloudy with '
                         'a temperature of 12.3 degrees Celsius.',
 'mode': 'auto',
 'tool_traces': [{'args': {'city': 'Warsaw'},
                  'output': {'city': 'Warsaw',
                             'summary': 'partly cloudy (demo only, not real '
                                        'data)',
                             'tempC': 12.3},
                  'output_preview': '{"city": "

RuntimeAnswer(answer='The current weather in Warsaw is partly cloudy with a temperature of 12.3 degrees Celsius.', citations=[], route=RouteInfo(used_rag=False, used_websearch=False, used_tools=True, used_long_term_memory=False, used_user_profile=True, strategy='llm_with_tools', extra={}), tool_calls=[ToolCallInfo(tool_name='calc_expression', arguments={'expression': '235*17'}, result_summary='{"expression": "235*17", "result": 3995}', success=True, error_message=None, extra={'raw_trace': {'tool': 'calc_expression', 'args': {'expression': '235*17'}, 'output_preview': '{"expression": "235*17", "result": 3995}', 'output': {'expression': '235*17', 'result': 3995}}})], stats=RuntimeStats(total_tokens=None, input_tokens=None, output_tokens=None, rag_tokens=None, websearch_tokens=None, tool_tokens=None, duration_ms=None, extra={}), raw_model_output=None, debug_trace={'session_id': 'demo-session-tools', 'user_id': 'demo-user-tools', 'config': {'llm_label': 'llm-adapter-ollama', 'embedding_lab